In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

# 1.Data Collection

In [ ]:
now = datetime.now()

ticker = "AAPL"
start = datetime(now.year - 10, now.month, now.day)
end = now 
df = yf.download(ticker, start=start, end=end, multi_level_index=False)

In [ ]:
df

# 2. Data Exploration & Visualization

In [ ]:
df.head()

In [ ]:
type(df)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df = df.reset_index()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12, 5));
plt.plot(df.Close);
plt.title(ticker);
plt.xlabel("Days");
plt.ylabel("Close Price");

# Feature Engineering

In [ ]:
# Calculating the moving average
temp_df = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
print(sum(temp_df[2:7]) / 5)

In [ ]:
df1 = pd.DataFrame(temp_df)
df1

In [ ]:
df1["MA_5"] = df1.rolling(5).mean()
df1

### 100 Days Moving Average

In [ ]:
df["MA_100"] = df.Close.rolling(100).mean()

In [ ]:
df.head()

In [ ]:
df.head(103)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df.Close)
plt.plot(df.MA_100, "r")
plt.title("100 Days Moving Avg.")
plt.xlabel("Days")
plt.ylabel("Price")

### 200 Days Moving Average

In [ ]:
df["MA_200"] = df.Close.rolling(200).mean()

In [ ]:
df.head()

In [ ]:
df.head(203)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df.Close)
plt.plot(df.MA_100, "r")
plt.plot(df.MA_200, "g")
plt.title("200 Days Moving Avg.")
plt.xlabel("Days")
plt.ylabel("Price")

### Percentage Changed Per Trading Session

In [ ]:
df["Percentage Changed"] = df.Close.pct_change()

In [ ]:
df[["Close", "Percentage Changed"]]

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df["Percentage Changed"])

# 4. Data Preprocessing

In [ ]:
df.shape

In [ ]:
# Splitting the data into Training & Testing datasets
data_training = pd.DataFrame(df.Close[0:int(len(df) * 0.7)])
data_testing = pd.DataFrame(df.Close[int(len(df) * 0.7):int(len(df))])

print(data_training)
print(data_testing)

In [ ]:
# Scalling down the data between 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaller = MinMaxScaler(feature_range=(0, 1))

In [ ]:
data_training_array = scaller.fit_transform(data_training)
data_training_array

In [ ]:
data_training_array.shape

# 5. Sequence Creation

In [ ]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
    x_train.append(data_training_array[i - 100:i])
    y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
y_train

In [ ]:
x_train

In [ ]:
y_train

# 6. Model Building

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
# ML Model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input

In [ ]:
model = Sequential()
model.add(Input(shape=(100, 1)))
model.add(LSTM(units=128, activation="tanh", return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(25))
model.add(Dense(1))

# 7. Model Training

In [ ]:
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(x_train, y_train, epochs=50)

In [ ]:
# Saving the model
model.save("stock_predicton_model.keras")

# 8. Preparing Test Data

In [ ]:
data_training.tail(100)

In [ ]:
data_testing

In [ ]:
past_100_days = data_training.tail(100)

In [ ]:
past_100_days

In [ ]:
final_df = pd.concat([past_100_days, data_testing], ignore_index=True)

In [ ]:
input_data = scaller.fit_transform(final_df)
input_data

In [ ]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i - 100:i])
    y_test.append(input_data[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
x_test

In [ ]:
y_test

# 9. Making Predictions

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
y_predicted = scaller.inverse_transform(y_predicted.reshape(-1, 1)).flatten()
y_test = scaller.inverse_transform(y_test.reshape(-1, 1)).flatten()

In [ ]:
y_predicted

In [ ]:
y_predicted

In [ ]:
plt.figure(figsize=(12, 6));
plt.plot(y_test, "b", label="Original Price");
plt.plot(y_predicted, "r", label="Predicted Price");
plt.xlabel("Days");
plt.ylabel("Price");
plt.legend();

# 10. Model Evaluation

In [ ]:
# Mean Squared Error
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_predicted)
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
# Root Mean Squared Error
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

In [ ]:
# R-Squared
r2= r2_score(y_test, y_predicted)
print(f"R-Squared (R2): {r2}")